In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip -qqq install git+https://www.github.com/keras-team/keras-contrib.git
!pip -qqq install --upgrade keras

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


import keras
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import tensorflow_hub as hub

from keras import backend as K

from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss

#from tensorflow.python.keras.models import Model, Input
#from tensorflow.python.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
#from tensorflow.python.keras.layers.merge import add


from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import Layer, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda


Using TensorFlow backend.
W0324 10:32:39.698910 140302001137536 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
train = pd.read_csv('/content/drive/My Drive/AV-Innoplexus/train 2.csv')
test = pd.read_csv('/content/drive/My Drive/AV-Innoplexus/test 2.csv')

In [5]:
train = train.drop(['Doc_ID'], axis = 1)
test = test.drop(['Doc_ID'], axis = 1)

train.head(5)

,id,Sent_ID,Word,tag
0,1,1,Obesity,O
1,2,1,in,O
2,3,1,Low-,O
3,4,1,and,O
4,5,1,Middle-Income,O


In [8]:
train.isnull().sum()

id         0
Sent_ID    0
Word       0
tag        0
dtype: int64

In [0]:
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [9]:
words = list(set(train["Word"].values))
n_words = len(words)
print("Number of words in the dataset: ", n_words)

tags = list(set(train["tag"].values))
print("Tags:", tags)
n_tags = len(tags)
print("Number of Labels: ", n_tags)

Number of words in the dataset:  184505
Tags: ['O', 'I-indications', 'B-indications']
Number of Labels:  3


In [0]:
## get pos_tags
'''
import nltk
nltk.download('averaged_perceptron_tagger')

def agg_func(x):
  for w, t in nltk.pos_tag([x]):
    return t

train['POS'] = train['Word'].apply(agg_func)
test['POS'] = test['Word'].apply(agg_func)'''

In [0]:
class GetSentences(object):
    
    def __init__(self, data, dataset_type):
        self.n_sent = 1
        self.data = data
        self.empty = False
        if dataset_type == 'train':
          agg_func = lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(), s['tag'].values.tolist())]
        else:
          agg_func = lambda s: [(w) for w in s['Word'].values.tolist()]
        self.grouped = self.data.groupby('Sent_ID').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = GetSentences(train, 'train')
train_sentences = getter.sentences


getter = GetSentences(test, 'test')
test_sentences = getter.sentences

In [11]:
len(train_sentences), len(test_sentences)

(191282, 125840)

In [0]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding

idx2word = {i: w for w, i in word2idx.items()}

tag2idx = {t: i+1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0

idx2tag = {i: w for w, i in tag2idx.items()}



In [0]:
from keras.preprocessing.sequence import pad_sequences
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in train_sentences]
# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=50, sequences=X, padding="post", value=word2idx["PAD"])

# Convert Tag/Label to tag_index
y = [[tag2idx[w[1]] for w in s] for s in train_sentences]
# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=50, sequences=y, padding="post", value=tag2idx["PAD"])


In [0]:

X = X[:102400]
y = y[:102400]


In [0]:
from keras.utils import to_categorical
# One-Hot encode
y = [to_categorical(i, num_classes=n_tags+1) for i in y]  # n_tags+1(PAD)


In [16]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_tr.shape, X_val.shape, np.array(y_tr).shape, np.array(y_val).shape

((81920, 50), (20480, 50), (81920, 50, 4), (20480, 50, 4))

In [0]:
'''import os
import pprint

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)
  
session = tf.Session(tpu_address)
#session.run(tf.contrib.tpu.initialize_system())
#session.run(tf.global_variables_initializer())
#session.run(tf.tables_initializer())'''

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [18]:
# Model definition
input = Input(shape=(50,))
model = Embedding(input_dim=n_words+2, output_dim=20, # n_words + 2 (PAD & UNK)
                  input_length=50, mask_zero=True)(input)  # default: 20-dim embedding
model = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(64, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


W0324 10:37:44.795073 140302001137536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0324 10:37:44.985996 140302001137536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 20)            3690140   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 128)           43520     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 64)            8256      
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 4)             284       
Total params: 3,742,200
Trainable params: 3,742,200
Non-trainable params: 0
_________________________________________________________________


In [0]:
'''
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
'''

In [19]:

#history = tpu_model.fit(X_tr, y_tr, epochs=3, batch_size=128*8, validation_data=(X_val, y_val))

history = model.fit(X_tr, np.array(y_tr), epochs=10, batch_size=128, validation_data=(X_val, np.array(y_val)))

Instructions for updating:
Use tf.cast instead.


W0324 10:38:35.063022 140302001137536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 81920 samples, validate on 20480 samples
Epoch 1/10
81920/81920 [==============================] - 272s 3ms/step - loss: 3.0770 - crf_viterbi_accuracy: 0.9714 - val_loss: 3.0859 - val_crf_viterbi_accuracy: 0.9848
Epoch 2/10
81920/81920 [==============================] - 271s 3ms/step - loss: 3.0133 - crf_viterbi_accuracy: 0.9870 - val_loss: 3.0708 - val_crf_viterbi_accuracy: 0.9875
Epoch 3/10
81920/81920 [==============================] - 270s 3ms/step - loss: 3.0025 - crf_viterbi_accuracy: 0.9890 - val_loss: 3.0651 - val_crf_viterbi_accuracy: 0.9884
Epoch 4/10
81920/81920 [==============================] - 269s 3ms/step - loss: 2.9968 - crf_viterbi_accuracy: 0.9904 - val_loss: 3.0617 - val_crf_viterbi_accuracy: 0.9894
Epoch 5/10
81920/81920 [==============================] - 269s 3ms/step - loss: 2.9934 - crf_viterbi_accuracy: 0.9913 - val_loss: 3.0611 - val_crf_viterbi_accuracy: 0.9892
Epoch 6/10
81920/81920 [==============================] - 270s 3ms/step - loss: 2.9911 - c

In [0]:
## prepare test_sentences

'''words = list(set(train["Word"].values))
n_words = len(words)
print("Number of words in the dataset: ", n_words)


word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding

idx2word = {i: w for w, i in word2idx.items()}'''


# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx.get(w, 1) for w in s] for s in test_sentences]
# Padding each sentence to have the same length
X_test = pad_sequences(maxlen=50, sequences=X_test, padding="post", value=word2idx["PAD"])



In [21]:
pred_c = model.predict(np.array(X_test), verbose = 1)
pred = np.argmax(pred_c, axis=-1)

125840/125840 [==============================] - 579s 5ms/step


In [0]:
# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in pred]

In [0]:
lengthlist = []
for s in test_sentences:
  lengthlist.append(len(s))

In [0]:
new_pred = []
for i, p in enumerate(pred_tag):
  newp = p[:lengthlist[i]]
  new_pred.append(newp)

In [0]:
flatlist = [item for sublist in new_pred for item in sublist]

In [0]:
test['tag'] = pd.Series(flatlist)

In [0]:
result = test.drop(['Word'], axis = 1)

In [32]:
result.head(5)

,id,Sent_ID,tag
0,4543834,191283,O
1,4543835,191283,O
2,4543836,191283,O
3,4543837,191283,O
4,4543838,191283,O


In [0]:
result.to_csv("/content/drive/My Drive/AV-Innoplexus/result_ner3.csv", sep = ",", index=False, columns=["id", "Sent_ID", "tag"])